# Importing required Python libraries/packages

In [1]:
import getpass
import oracledb
import pandas as pd
from datetime import date

# Initialising **thick mode** for Oracle DB - for connection with the local DB instance.

In [2]:
oracledb.init_oracle_client(lib_dir='/opt/oracle/product/21c/dbhomeXE/lib')

# Establishing connection to the DB, as *SYS* user, in *SYSDBA* role

In [3]:
connection = oracledb.connect(
	mode=oracledb.AUTH_MODE_SYSDBA,
	dsn="localhost/XEPDB1"
)

# Creating a **cursor** to interact with the DB instance

In [4]:
cursor = connection.cursor()

## Dropping existing tables

In [5]:
def table_exists(table_name):
    cursor.execute(f"SELECT COUNT(*) FROM USER_TABLES WHERE TABLE_NAME = UPPER('{table_name}')")
    count = cursor.fetchone()[0]
    return count > 0

tables = ["bill", "appointment", "treatment", "room", "doctor", "patient"] # children -> parent order.
for table in tables:
    if table_exists(table):
        try:
            print(f"Dropping table {table}...")
            cursor.execute(f"DROP TABLE {table} CASCADE CONSTRAINTS PURGE")
            print(f"Table {table} dropped successfully.")
        except oracledb.DatabaseError as e:
            print(f"Error dropping table {table}: {e}")
    else:
        print(f"Table {table} does not exist.")

Dropping table bill...
Table bill dropped successfully.
Dropping table appointment...
Table appointment dropped successfully.
Dropping table treatment...
Table treatment dropped successfully.
Dropping table room...
Table room dropped successfully.
Dropping table doctor...
Table doctor dropped successfully.
Dropping table patient...
Table patient dropped successfully.


# Creating the tables

## Patient table

In [6]:
# cursor.execute("drop table patient")

cursor.execute("""
create table patient(
    patient_id number primary key,
    name varchar2(100),
    address varchar2(255),
    date_of_birth date,
    phone varchar2(15)
    )
""")

## Doctor table

In [7]:
#cursor.execute("drop table doctor")

cursor.execute("""
create table doctor(
    doctor_id number primary key,
    name varchar2(100),
    specializations varchar2(255),
    experience_years number,
    contact varchar2(15)
    )
""")

## Room table

In [8]:
#cursor.execute("drop table room")

cursor.execute("""
create table room(
    room_id number primary key,
    room_type varchar2(50),
    capacity number,
    status varchar2(50)
)
""")

## Treatment table

In [9]:
#cursor.execute("drop table treatment")

cursor.execute("""
create table treatment(
    treatment_id number primary key,
    patient_id number,
    doctor_id number,
    treatment_type varchar2(100),
    start_date date,
    end_date date,
    foreign key(patient_id) references patient(patient_id),
    foreign key(doctor_id) references doctor(doctor_id)
)
""")

## Appointment table

In [10]:
# cursor.execute("drop table appointment")

cursor.execute("""
create table appointment(
    appointment_id number primary key,
    patient_id number,
    doctor_id number,
    room_id number,
    room_type varchar(35),
    appointment_date date,
    appointment_time varchar2(10),
    foreign key(patient_id) references patient(patient_id),
    foreign key(doctor_id) references doctor(doctor_id),
    foreign key(room_id) references room(room_id)
)
""")

## Bill table

In [18]:
# cursor.execute("drop table bill")

cursor.execute("""
create table bill(
    bill_id number primary key,
    patient_id number,
    treatment_id number,
    total_amount number,
    date_issued date,
    foreign key(treatment_id) references treatment(treatment_id)
)
""")

# Inserting records

## Patient table

In [11]:
patients = [
    (1, 'Aarav', '123, ABC St', date(1994, 5, 5), '1234567890'),
    (2, 'Priya', '45, DEF St', date(1995, 8, 12), '0987654321'),
    (3, 'Ravi', '78, GHI St', date(1994, 3, 3), '5647382910'),
    (4, 'Simran', '56, JKL St', date(1996, 2, 15), '4321567890')
]

cursor.executemany("""
    insert into patient values(:1, :2, :3, :4, :5)
    """, patients)

cursor.execute("select * from patient")
results = cursor.fetchall()
columns = [col[0] for col in cursor.description]

df = pd.DataFrame(results, columns=columns)
df

,PATIENT_ID,NAME,ADDRESS,DATE_OF_BIRTH,PHONE
0,1,Aarav,"123, ABC St",1994-05-05,1234567890
1,2,Priya,"45, DEF St",1995-08-12,0987654321
2,3,Ravi,"78, GHI St",1994-03-03,5647382910
3,4,Simran,"56, JKL St",1996-02-15,4321567890


## Doctor table

In [ ]:
doctors = [
    (1, 'Dr. A', 'Cardiology', 15, '1231231234'),
    (2, 'Dr. B', 'Neurology', 10, '3213214321'),
    (3, 'Dr. C', 'Orthopedics', 20, '6546546543'),
    (4, 'Dr. D', 'Dermatology', 8, '7897897890'),
]

cursor.executemany("""
    insert into doctor values(:1, :2, :3, :4, :5)
    """, doctors)

cursor.execute("select * from doctor")
results = cursor.fetchall()
columns = [col[0] for col in cursor.description]

df = pd.DataFrame(results, columns=columns)
df

## Treatment table

In [14]:
treatments = [
    (1, 1, 1, 'Heart Surgery', date(2024, 1, 1), date(2024, 1, 10)),
    (2, 2, 2, 'Neurological Checkup', date(2024, 2, 5), date(2024, 2, 6)),
    (3, 3, 3, 'Orthopedic Surgery', date(2024, 3, 3), date(2024, 3, 10)),
    (4, 4, 4, 'Skin Treatment', date(2024, 4, 4), date(2024, 4, 8))
]

cursor.executemany("""
    insert into treatment values(:1, :2, :3, :4, :5, :6)
    """, treatments)

cursor.execute("select * from treatment")
results = cursor.fetchall()
columns = [col[0] for col in cursor.description]

df = pd.DataFrame(results, columns=columns)
df

,TREATMENT_ID,PATIENT_ID,DOCTOR_ID,TREATMENT_TYPE,START_DATE,END_DATE
0,1,1,1,Heart Surgery,2024-01-01,2024-01-10
1,2,2,2,Neurological Checkup,2024-02-05,2024-02-06
2,3,3,3,Orthopedic Surgery,2024-03-03,2024-03-10
3,4,4,4,Skin Treatment,2024-04-04,2024-04-08


## Room table

In [15]:
rooms = [
    (101, 'ICU', 2, 'Occupied'),
    (102, 'General', 4, 'Available'),
    (103, 'ICU', 1, 'Available'),
    (104, 'General', 4, 'Occupied')
]

cursor.executemany("""
    insert into room values(:1, :2, :3, :4)
    """, rooms)

cursor.execute("select * from room")
results = cursor.fetchall()
columns = [col[0] for col in cursor.description]

df = pd.DataFrame(results, columns=columns)
df

,ROOM_ID,ROOM_TYPE,CAPACITY,STATUS
0,101,ICU,2,Occupied
1,102,General,4,Available
2,103,ICU,1,Available
3,104,General,4,Occupied


## Appointment table

In [16]:
appointments = [
    (1, 1, 1, 101, 'General', date(2024, 6, 1), '10:00'),
    (2, 2, 1, 102, 'ICU', date(2024, 7, 5), '11:00'),
    (3, 3, 3, 103, 'General', date(2023, 11, 3), '12:00'),
    (4, 4, 4, 104, 'General', date(2023, 12, 4), '13:00'),
]

cursor.executemany("""
    insert into appointment values(:1, :2, :3, :4, :5, :6, :7)
    """, appointments)

cursor.execute("select * from appointment")
results = cursor.fetchall()
columns = [col[0] for col in cursor.description]

df = pd.DataFrame(results, columns=columns)
df

,APPOINTMENT_ID,PATIENT_ID,DOCTOR_ID,ROOM_ID,ROOM_TYPE,APPOINTMENT_DATE,APPOINTMENT_TIME
0,1,1,1,101,General,2024-06-01,10:00
1,2,2,1,102,ICU,2024-07-05,11:00
2,3,3,3,103,General,2023-11-03,12:00
3,4,4,4,104,General,2023-12-04,13:00


## Bill table

In [19]:
bills = [
    (1, 1, 1, 50000, date(2024, 1, 10)),
    (2, 2, 2, 15000, date(2024, 2, 6)),
    (3, 3, 3, 75000, date(2024, 3, 10)),
    (4, 4, 4, 10000, date(2024, 4, 8))
]

cursor.executemany("""
    insert into bill values(:1, :2, :3, :4, :5)
    """, bills)

cursor.execute("select * from bill")
results = cursor.fetchall()
columns = [col[0] for col in cursor.description]

df = pd.DataFrame(results, columns=columns)
df

,BILL_ID,PATIENT_ID,TREATMENT_ID,TOTAL_AMOUNT,DATE_ISSUED
0,1,1,1,50000,2024-01-10
1,2,2,2,15000,2024-02-06
2,3,3,3,75000,2024-03-10
3,4,4,4,10000,2024-04-08


# Solving the questions

## Create Room table with attributes Room_ID, Room_type, capacity, status

In [ ]:
cursor.execute("""
create table room(
    room_id number primary key,
    room_type varchar2(50),
    capacity number,
    status varchar2(50)
)
""")

## Insert sample records for the patients, with all non-null values

In [ ]:
patients = [
    (1, 'Aarav', '123, ABC St', date(1994, 5, 5), '1234567890'),
    (2, 'Priya', '45, DEF St', date(1995, 8, 12), '0987654321'),
    (3, 'Ravi', '78, GHI St', date(1994, 3, 3), '5647382910'),
    (4, 'Simran', '56, JKL St', date(1996, 2, 15), '4321567890')
]

cursor.executemany("""
    insert into patient values(:1, :2, :3, :4, :5)
    """, patients)


## Find total capacity of all ICU rooms among all the rooms. 

In [21]:
cursor.execute("""
    select sum(capacity) as total_icu_capacity from room
    where room_type='ICU'
""")

results = cursor.fetchall()
columns = [col[0] for col in cursor.description]

df = pd.DataFrame(results, columns=columns)
df

,TOTAL_ICU_CAPACITY
0,3


## Display treatment details for heart surgeries 

In [22]:
cursor.execute("""
    SELECT * 
    FROM Treatment 
    WHERE Treatment_Type = 'Heart Surgery'
""")

results = cursor.fetchall()
columns = [col[0] for col in cursor.description]

df = pd.DataFrame(results, columns=columns)
df

,TREATMENT_ID,PATIENT_ID,DOCTOR_ID,TREATMENT_TYPE,START_DATE,END_DATE
0,1,1,1,Heart Surgery,2024-01-01,2024-01-10


## Rename Specializations to Spec for doctors

In [23]:
cursor.execute("""
    alter table doctor
    rename column specializations
    to spec
""")

cursor.execute("select * from doctor")
results = cursor.fetchall()
columns = [col[0] for col in cursor.description]

df = pd.DataFrame(results, columns=columns)
df

,DOCTOR_ID,NAME,SPEC,EXPERIENCE_YEARS,CONTACT
0,1,Dr. A,Cardiology,15,1231231234
1,2,Dr. B,Neurology,10,3213214321
2,3,Dr. C,Orthopedics,20,6546546543
3,4,Dr. D,Dermatology,8,7897897890


## Add foreign key constraint: patient_id in Bill references patient_id in Patient.

In [24]:
cursor.execute("""
    alter table bill
    add constraint fk_patient
    foreign key(patient_id) 
    references patient(patient_id)
    """)
cursor.execute("select * from bill")
results = cursor.fetchall()
columns = [col[0] for col in cursor.description]

df = pd.DataFrame(results, columns=columns)
df

,BILL_ID,PATIENT_ID,TREATMENT_ID,TOTAL_AMOUNT,DATE_ISSUED
0,1,1,1,50000,2024-01-10
1,2,2,2,15000,2024-02-06
2,3,3,3,75000,2024-03-10
3,4,4,4,10000,2024-04-08


## Find the number of appointments per (for each) doctor

In [25]:
cursor.execute("""
    select d.name as doctor_name, count(a.appointment_id) as no_of_appointments
    from appointment a
    join doctor d on
    a.doctor_id = d.doctor_id
    group by d.name
    """)
results = cursor.fetchall()
columns = [col[0] for col in cursor.description]

df = pd.DataFrame(results, columns=columns)
df

,DOCTOR_NAME,NO_OF_APPOINTMENTS
0,Dr. A,2
1,Dr. C,1
2,Dr. D,1


## Select all patients who had any appointment in the last 6 months. 

In [26]:
cursor.execute("""
select p.name, a.appointment_date
from patient p join appointment a
on p.patient_id = a.patient_id
where appointment_date >= add_months(trunc(sysdate), -6)
""")

results = cursor.fetchall()
columns = [col[0] for col in cursor.description]

df = pd.DataFrame(results, columns=columns)
df

,NAME,APPOINTMENT_DATE
0,Aarav,2024-06-01
1,Priya,2024-07-05


## Make the default value of status 'Available' in Room, and add a check constraint to ensure Capacity is positive. 

In [27]:
cursor.execute("""
    alter table room 
    modify status 
    default 'Available'
    """)

cursor.execute("""
    alter table room
    add constraint chk_capacity_positive
    check (capacity > 0)
    """)

## Find the total bill amount for each patient. 

In [28]:
cursor.execute("""
    select p.patient_id, p.name as patient_name, sum(b.total_amount) as total_amount
    from patient p left join bill b on
    p.patient_id = b.patient_id
    group by p.patient_id, p.name
    order by p.name
    """)
results = cursor.fetchall()
columns = [col[0] for col in cursor.description]

df = pd.DataFrame(results, columns=columns)
df

,PATIENT_ID,PATIENT_NAME,TOTAL_AMOUNT
0,1,Aarav,50000
1,2,Priya,15000
2,3,Ravi,75000
3,4,Simran,10000


# Closing up the cursor and connection

In [29]:
connection.commit()
cursor.close()
connection.close()